### Canned Estimators

Source: https://github.com/random-forests/tensorflow-workshop/blob/master/examples/04_canned_estimators.ipynb

In this notebook we'll demonstrate how to use two Canned Estimators (these encapsulate the lower-level TensorFlow code we've seen so far, and use an API loosely inspired by [scikit-learn](http://scikit-learn.org). There are several advantages to Canned Estimators.

* If you're using Estimators, you won't have to manage Sessions, or write your own logic for TensorBoard, or for saving and loading checkpoints.


* You'll get out-of-the-box distributed training (of course, you will have to take care to read your data efficiently, and set up a cluster).

Here, we'll read data using [input functions](https://www.tensorflow.org/api_docs/python/tf/estimator/inputs/numpy_input_fn), which are appropriate for in-memory data. 

* These provide batching and other features for you, so you don't have to write that code yourself.


* In the strucuted data notebook, we'll use the new [Dataset API](https://www.tensorflow.org/programmers_guide/datasets) - which is a popular abstraction, and a great way to efficiently read and pre-process large datasets efficiently.

Although the Estimators we'll use here are relative simple (a LinearClassifier, and a Fully Connected Deep Neural Network), we also provide more interesting ones (including for [TensorFlow Wide and Deep](https://www.tensorflow.org/tutorials/wide_and_deep). I'm also excited that additional Estimators are on their way - stay tuned in the upcoming months.

Also note that Estimators log quite a lot of output

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

# It will be downloaded to './data' if you don't already have a local copy.
mnist = input_data.read_data_sets('./data', one_hot=False)

x_train = mnist.train.images
y_train = mnist.train.labels.astype(np.int32)
x_test = mnist.test.images
y_test = mnist.test.labels.astype(np.int32)

Extracting ./data/train-images-idx3-ubyte.gz
Extracting ./data/train-labels-idx1-ubyte.gz
Extracting ./data/t10k-images-idx3-ubyte.gz
Extracting ./data/t10k-labels-idx1-ubyte.gz


Here's our input function. 

* By setting ```num_epochs``` to ```None```, we'll loop over the data indefinitely so we can train for as long as we like.
* The default ```batch_size``` is ```128```, but you can provide a different parameter if you like.

You can read more about the numpy input function [here](https://www.tensorflow.org/api_docs/python/tf/estimator/inputs/numpy_input_fn). We also provide a nice one for [Pandas](https://www.tensorflow.org/api_docs/python/tf/estimator/inputs/pandas_input_fn), more on that later.

In [3]:
train_input = tf.estimator.inputs.numpy_input_fn(
    {'x': x_train},
    y_train, 
    num_epochs=None, # repeat forever
    shuffle=True # 
)

In [4]:
test_input = tf.estimator.inputs.numpy_input_fn(
    {'x': x_test},
    y_test,
    num_epochs=1, # loop through the dataset once
    shuffle=False # don't shuffle the test data
)

In [5]:
# define the features for our model
# the names must match the input function
feature_spec = [tf.feature_column.numeric_column('x', shape=784)]

Here, we'll create a ```LinearClassifier``` - this is identical to our Softmax (aka, multiclass logistic regression model) from the second notebok.

In [6]:
estimator = tf.estimator.LinearClassifier(feature_spec,
                                          n_classes=10,
                                          model_dir="./graphs/04_canned/linear")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './graphs/04_canned/linear', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}


In [7]:
# I've arbitrarily decided to train for 1000 steps
estimator.train(train_input, steps=1000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ./graphs/04_canned/linear/model.ckpt.
INFO:tensorflow:loss = 294.731, step = 1
INFO:tensorflow:global_step/sec: 423.716
INFO:tensorflow:loss = 78.2788, step = 101 (0.237 sec)
INFO:tensorflow:global_step/sec: 570.363
INFO:tensorflow:loss = 34.4535, step = 201 (0.175 sec)
INFO:tensorflow:global_step/sec: 669.707
INFO:tensorflow:loss = 32.9108, step = 301 (0.149 sec)
INFO:tensorflow:global_step/sec: 718.933
INFO:tensorflow:loss = 59.5948, step = 401 (0.139 sec)
INFO:tensorflow:global_step/sec: 722.34
INFO:tensorflow:loss = 39.6534, step = 501 (0.138 sec)
INFO:tensorflow:global_step/sec: 717.417
INFO:tensorflow:loss = 31.2589, step = 601 (0.139 sec)
INFO:tensorflow:global_step/sec: 713.803
INFO:tensorflow:loss = 26.0216, step = 701 (0.140 sec)
INFO:tensorflow:global_step/sec: 658.9
INFO:tensorflow:loss = 44.4672, step = 801 (0.152 sec)
INFO:tensorflow:global_step/sec: 709.915
INFO:tensorflow:loss = 38

In [8]:
# We should see about 90% accuracy here.
evaluation = estimator.evaluate(input_fn=test_input)
print(evaluation)

INFO:tensorflow:Starting evaluation at 2017-10-01-17:32:46
INFO:tensorflow:Restoring parameters from ./graphs/04_canned/linear/model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2017-10-01-17:32:47
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.9244, average_loss = 0.282789, global_step = 1000, loss = 35.7961
{'accuracy': 0.92439997, 'average_loss': 0.28278899, 'loss': 35.796074, 'global_step': 1000}


Here's how you would print individual predictions.

In [9]:
MAX_TO_PRINT = 5

# This returns a generator object
predictions = estimator.predict(input_fn=test_input)
i = 0
for p in predictions:
    true_label = y_test[i]
    predicted_label = p['class_ids'][0]
    print("Example %d. True: %d, Predicted: %d" % (i, true_label, predicted_label))
    i += 1
    if i == MAX_TO_PRINT: break

INFO:tensorflow:Restoring parameters from ./graphs/04_canned/linear/model.ckpt-1000
Example 0. True: 7, Predicted: 7
Example 1. True: 2, Predicted: 2
Example 2. True: 1, Predicted: 1
Example 3. True: 0, Predicted: 0
Example 4. True: 4, Predicted: 4


Here's how easy it is to switch the model to a fully connected DNN.

In [10]:
estimator = tf.estimator.DNNClassifier(
    hidden_units=[256], # we will arbitrarily use two layers
    feature_columns=feature_spec,
    n_classes=10,
    model_dir="./graphs/04_canned/deep")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './graphs/04_canned/deep', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}


In [11]:
# I've arbitrarily decided to train for 2000 steps
estimator.train(train_input, steps=2000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ./graphs/04_canned/deep/model.ckpt.
INFO:tensorflow:loss = 307.062, step = 1
INFO:tensorflow:global_step/sec: 150.782
INFO:tensorflow:loss = 33.6069, step = 101 (0.665 sec)
INFO:tensorflow:global_step/sec: 174.718
INFO:tensorflow:loss = 17.999, step = 201 (0.572 sec)
INFO:tensorflow:global_step/sec: 200.66
INFO:tensorflow:loss = 14.8136, step = 301 (0.498 sec)
INFO:tensorflow:global_step/sec: 203.317
INFO:tensorflow:loss = 15.2711, step = 401 (0.492 sec)
INFO:tensorflow:global_step/sec: 203.068
INFO:tensorflow:loss = 6.86588, step = 501 (0.493 sec)
INFO:tensorflow:global_step/sec: 203.259
INFO:tensorflow:loss = 6.40926, step = 601 (0.492 sec)
INFO:tensorflow:global_step/sec: 184.367
INFO:tensorflow:loss = 10.7463, step = 701 (0.542 sec)
INFO:tensorflow:global_step/sec: 158.124
INFO:tensorflow:loss = 10.4022, step = 801 (0.633 sec)
INFO:tensorflow:global_step/sec: 189.523
INFO:tensorflow:loss = 10.

In [12]:
# Expect accuracy around 97%
evaluation = estimator.evaluate(input_fn=test_input)
print(evaluation)

INFO:tensorflow:Starting evaluation at 2017-10-01-17:33:01
INFO:tensorflow:Restoring parameters from ./graphs/04_canned/deep/model.ckpt-2000
INFO:tensorflow:Finished evaluation at 2017-10-01-17:33:02
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.9752, average_loss = 0.0790281, global_step = 2000, loss = 10.0036
{'accuracy': 0.9752, 'average_loss': 0.079028137, 'loss': 10.003562, 'global_step': 2000}


If you like, you can compare these runs with TensorBoard.

``` $ tensorboard --logdir=graphs/04_canned/ ```